<a href="https://colab.research.google.com/github/datascience-uniandes/data-quality-tutorial/blob/master/data-quality-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality tutorial

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
Dataset: Homicides Colombia  
Source: [datos.gov.co](datos.gov.co)
  
Last update: September, 2022

In [1]:
!pip install pylev

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Importing required libraries

import re
from random import randint
from datetime import datetime

import numpy as np
import pandas as pd

import pylev

In [3]:
# Parameter for showing all columns when printing a dataframe
pd.set_option('display.max_columns', None)

In [5]:
from google.colab import drive

In [6]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [54]:
# Loading data
df = pd.read_csv("/content/gdrive/MyDrive/Quiz Ciencia de Datos Aplicada/homicides.csv")

In [55]:
# Printing the dataset dimensions
df.shape

(12400, 22)

In [56]:
# Printing column data types
df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [57]:
type(df["FECHA"])

pandas.core.series.Series

In [58]:
df.head()

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,01/01/2021 12:00:00 AM,ANTIOQUIA,AMAGÁ,Jueves,6:00,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,01/01/2021 12:00:00 AM,ANTIOQUIA,BARBOSA,Jueves,9:00,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,01/01/2021 12:00:00 AM,ANTIOQUIA,EL BAGRE,Jueves,19:00,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,01/01/2021 12:00:00 AM,ANTIOQUIA,JARDÍN,Jueves,11:20,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Juees,15:00,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co


In [59]:
df.columns

Index(['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'DIA', 'HORA', 'BARRIO', 'ZONA',
       'CLASE DE SITIO', 'ARMA O MEDIO', 'MOVIL VICTIMA', 'MOVIL AGRESOR',
       'EDAD', 'GENERO', 'ESTADO CIVIL', 'CLASE EMPLEADO', 'PROFESION',
       'ESCOLARIDAD', 'PAIS NACE', 'CODIGO DANE', 'AÑO DE NACIMIENTO',
       'CÉDULA', 'CORREO'],
      dtype='object')

In [60]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [61]:
# Applying the validation to all values in column
df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR INTENTIONALLY

ValueError: ignored

In [62]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [63]:
# Using the function for validation
df.loc[df["FECHA"].apply(error_in_format)]

# THE LAST ROW IS CAUSING THE ERROR

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
12399,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
# Deleting a row by its index
df.drop(12399, inplace = True)

In [65]:
# Trying to parse the datetime string again
df["FECHA"] = df["FECHA"].apply(dateparse)

In [66]:
# Counting homicides by hour
df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

12    12399
Name: FECHA, dtype: int64

In [67]:
# The homicide hour is available in a different column!
# Merging both columns
df["FECHA"] = df["FECHA"].astype(str).apply(lambda x: x[:11]) + df["HORA"]

In [68]:
del df["HORA"]

In [69]:
df["FECHA"].head()

0     2021-01-01 6:00
1     2021-01-01 9:00
2    2021-01-01 19:00
3    2021-01-01 11:20
4    2021-01-01 15:00
Name: FECHA, dtype: object

In [70]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [71]:
# Applying the expression
df["FECHA"] = df["FECHA"].apply(dateparse)

In [72]:
df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [73]:
df.head()

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 06:00:00,ANTIOQUIA,AMAGÁ,Jueves,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co


In [74]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [75]:
# Finding values not matching with the dictionary
df.loc[~df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary</span>

<span style="color:red">Hint: You can use the replace() pandas function</span>

In [76]:
df["DEPARTAMENTO"]=df["DEPARTAMENTO"].replace(["SAN ANDRÉS","N. DE SANTANDER"],["SAN ANDRÉS Y PROVIDENCIA","NORTE DE SANTANDER"])
# Finding values not matching with the dictionary
df.loc[~df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()
#Esto valida que las observaciones fueron reemplazadas de manera satisfactoria

array([], dtype=object)

In [77]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = df.loc[df["CÉDULA"].duplicated(keep = False)]

In [78]:
duplicates_by_cedula.shape

(1627, 21)

In [79]:
# Showing some examples
duplicates_by_cedula.sort_values("CÉDULA", ascending = True).head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4041,2021-05-03 23:30:00,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com
3810,2021-04-26 03:00:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 05:30:00,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co
10796,2021-11-20 03:00:00,VALLE,CALI (CT),Viernes,POTRERO GRANDE E21,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1994.0,10-079,iknnoj8430@unidatos.edu.co
2386,2021-03-14 02:00:00,META,FUENTE DE ORO,Sábado,VEREDA PUERTO NUEVO,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA BLANCA,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,50287000.0,1973.0,10-255,afompq7113@unidatos.edu.co
138,2021-01-02 06:20:00,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

In [80]:
#Removing rows with the same First Name
df.drop_duplicates(subset="CÉDULA",keep=False, inplace=True)

duplicates_by_cedula = df.loc[df["CÉDULA"].duplicated(keep = False)]
duplicates_by_cedula.shape
duplicates_by_cedula.sort_values("CÉDULA", ascending = True).head(6)
#Esto valida que no existen cédulas duplicadas

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO


In [81]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = df.loc[df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [82]:
cedula_malformed.shape

(28, 21)

In [83]:
cedula_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,680-21,oohghd8899@gmail.com
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,975-31,ohbqrk3631@unidatos.edu.co
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,348-66,pdkqur8407@unidatos.edu.co
3409,2021-04-13 23:20:00,CÓRDOBA,SAHAGÚN,Lunes,CORREGIMIENTO DE BAJO GRANDE,URBANA,BILLARES,CONTUNDENTES,A PIE,A PIE,49.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,23660000.0,1966.0,496-18,fdbbeo6751@unidatos.edu.co
3588,2021-04-19 20:00:00,BOLÍVAR,CARTAGENA (CT),Domingo,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,NO REPORTADO,NO REPORTADO,13001000.0,1970.0,188-03,diebuo5651@unidatos.edu.co


<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

In [84]:
df["CÉDULA"]=df["CÉDULA"].replace({'-':''}, regex=True) #Quitamos todos los strings -
df["CÉDULA"] = df["CÉDULA"].str[:2]+'-'+df["CÉDULA"].str[2:] #Agregamos los strings - en la tercera posición
df.head(6)


,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
5,2021-01-01 00:47:00,ANTIOQUIA,MEDELLÍN (CT),Juees,CORDOBA CNO REPORTADO7,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1973.0,54-450,grlcsj8696@unidatos.edu.co
6,2021-01-01 11:00:00,ANTIOQUIA,MEDELLÍN (CT),Jueves,CRISTO REY CNO REPORTADO15,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1990.0,45-052,pkdakc6682@gmail.com


In [85]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = df.loc[df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]
cedula_malformed.shape

(0, 21)

In [40]:
cedula_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO


<span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

In [86]:
# Using regular expressions for validating if "CORREO" values match the pattern  before the @, only .edu.co and .com
mail_malformed = df.loc[df["CORREO"].apply(lambda x: (re.search(".edu.co$|com$", x) is None))]
mail_malformed.shape

(879, 21)

In [87]:
mail_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
36,2021-01-01 08:00:00,CAUCA,PATÍA,Jueves,C/MIENTO PIEDRASENTADA,RURAL,CARRETERA,ARMA DE FUEGO,VEHICULO,A PIE,26.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19532000.0,1989.0,69-084,correo4714@colombia.gov.co
55,2021-01-01 01:30:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,ÁLVARO BERNAL SEGURA E-19,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,25.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1990.0,35-260,correo7285@colombia.gov.co
84,2021-01-01 08:40:00,NARIÑO,SAN ANDRES DE TUMACO,Jueces,PANAMA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,26.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,52835000.0,1989.0,55-611,correo8342@colombia.gov.co
104,2021-01-01 03:40:00,VALLE,CALI (CT),Jueces,12 DE OCTUBRE E12,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,49.0,MASCULINO,CASADO,EMPLEADO PUBLICO,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,66.0,87-287,correo6479@colombia.gov.co
122,2021-01-01 12:00:00,VALLE,PALMIRA,Juees,LORETO,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,76520000.0,1991.0,81-854,correo5547@colombia.gov.co
131,2021-01-02 22:20:00,CALDAS,MANIZALES (CT),Viernes,SAN SEBASTIAN,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,17001000.0,1997.0,90-886,correo7410@colombia.gov.co


In [88]:
df= df.loc[df["CORREO"].apply(lambda x: True if re.search(".edu.co$|com$", x) else False)]
df


,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
5,2021-01-01 00:47:00,ANTIOQUIA,MEDELLÍN (CT),Juees,CORDOBA CNO REPORTADO7,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1973.0,54-450,grlcsj8696@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12393,2021-12-31 23:30:00,VALLE,CALI (CT),Jueces,LAS ORQUIDEAS E14,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,43.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1972.0,36-531,cqlqhl7009@unidatos.edu.co
12394,2021-12-31 15:42:00,VALLE,EL CERRITO,Jueves,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,53.0,FEMENINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,76248000.0,1962.0,39-658,nggiso7664@gmail.com
12396,2021-12-31 22:35:00,VALLE,PALMIRA,Jueves,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1997.0,25-368,fktatq9245@unidatos.edu.co
12397,2021-12-31 14:30:00,VALLE,ZARZAL,Jueves,URB. EL ZARZALITO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76895000.0,1988.0,62-735,gafkup6393@gmail.com


In [89]:
mail_malformed = df.loc[df["CORREO"].apply(lambda x: (re.search(".edu.co$|com$", x) is None))]
mail_malformed.shape

(0, 21)

In [90]:
mail_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO


In [91]:
# Showing different values for "DIA" column
df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Ssbado', 'Domingo', 'Domungo', 'Doningo',
       'domungo', 'lune', 'Lunes', 'Luns', 'Lumes', 'kunes', 'Lnues',
       'Martes', 'Marte', 'mates', 'Mates', 'Miércoles', 'Miwrcoles',
       'Mircoles', 'Voernes', 'Sávado', 'domnigo', 'Maryes', 'Miercoles',
       'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'], dtype=object)

In [93]:
# Calculating the distance between two words using the Levenshtein method
pylev.levenshtein('sábado', 'sabaod')

3

In [94]:
pylev.levenshtein('sábado', 'viernes')

7

In [95]:
def change(x):
    if x in ['Lunes','lune','Luns','Lumes','kunes','Lnues']:
        return 'Lunes'
    if x in ['Martes','Marte','mates','Mates','Maryes','Msrtes']:
        return 'Martes'
    if x in ['Miércoles','Miwrcoles','Mircoles','Miercoles','miércles','Mirrcoles']:
        return 'Miércoles'
    if x in ['Jueves','Juees','Jueces','juves','Juevrs']:
        return 'Jueves'
    if x in ['Viernes','Viermes', 'iernes', 'virnes', 'Vierens','Voernes']:
        return 'Viernes'
    if x in ['Sábado','Sabadi','Sabado','sábad','Ssbado','Sávado']:
        return 'Sábado'
    if x in ['Domingo', 'Domungo', 'Doningo','domungo','domnigo','Dominog']:
        return 'Domingo'   


    
        

In [96]:
df["DIA"]=df["DIA"].apply(lambda x: change(x))
df


,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Jueves,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
5,2021-01-01 00:47:00,ANTIOQUIA,MEDELLÍN (CT),Jueves,CORDOBA CNO REPORTADO7,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1973.0,54-450,grlcsj8696@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12393,2021-12-31 23:30:00,VALLE,CALI (CT),Jueves,LAS ORQUIDEAS E14,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,43.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1972.0,36-531,cqlqhl7009@unidatos.edu.co
12394,2021-12-31 15:42:00,VALLE,EL CERRITO,Jueves,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,53.0,FEMENINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,76248000.0,1962.0,39-658,nggiso7664@gmail.com
12396,2021-12-31 22:35:00,VALLE,PALMIRA,Jueves,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1997.0,25-368,fktatq9245@unidatos.edu.co
12397,2021-12-31 14:30:00,VALLE,ZARZAL,Jueves,URB. EL ZARZALITO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76895000.0,1988.0,62-735,gafkup6393@gmail.com


<span style="color:red">TODO: Create a function to fix the digitation errors for "DIA" column.</span>

In [97]:
df["DIA"].unique()

array(['Jueves', 'Viernes', 'Sábado', 'Domingo', 'Lunes', 'Martes',
       'Miércoles'], dtype=object)

In [98]:
df[["AÑO DE NACIMIENTO", "EDAD"]].head(10)

,AÑO DE NACIMIENTO,EDAD
1,1985.0,30.0
2,1982.0,33.0
3,1975.0,40.0
4,1949.0,66.0
5,1973.0,42.0
6,1990.0,25.0
7,1978.0,37.0
9,1985.0,30.0
10,1974.0,41.0
11,1985.0,30.0


<span style="color:red">TODO: Fix the "AÑO DE NACIMIENTO" column using the column "EDAD".</span>

In [99]:
currentDateTime = datetime.now()
currentDateTime.year

df["AÑO DE NACIMIENTO"]=currentDateTime.year-df["EDAD"]
df[["AÑO DE NACIMIENTO", "EDAD"]].head(10)

,AÑO DE NACIMIENTO,EDAD
1,1992.0,30.0
2,1989.0,33.0
3,1982.0,40.0
4,1956.0,66.0
5,1980.0,42.0
6,1997.0,25.0
7,1985.0,37.0
9,1992.0,30.0
10,1981.0,41.0
11,1992.0,30.0
